# News API

A Python client for the [News API](https://newsapi.org/docs/).


## General

This is a Python client library for [News API V2](https://newsapi.org/).
The functions and methods for this library should mirror the
endpoints specified by the News API [documentation](https://newsapi.org/docs/endpoints).


## Installation

Installation for the package can be done via `pip`:

In [ ]:
%pip install newsapi-python
%pip install opencv-python
%pip install mediapipe
%pip install screeninfo
%pip install cvzone
%pip install matplotlib
%pip install pandas

## Usage

After installation, import the client class into your project:

In [ ]:
from newsapi import NewsApiClient

Initialize the client with your API key:

In [ ]:
import os

api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])


### Endpoints

An instance of `NewsApiClient` has three instance methods corresponding to three News API endpoints.

#### Top Headlines

Use `.get_top_headlines()` to pull from the [`/top-headlines`](https://newsapi.org/docs/endpoints/top-headlines) endpoint:

In [ ]:
api.get_top_headlines(sources='bbc-news')

#### Everything

Use `.get_everything()` to pull from the [`/everything`](https://newsapi.org/docs/endpoints/everything) endpoint:

In [ ]:
api.get_everything(q='bitcoin')

#### Sources

Use `.get_sources()` to pull from the [`/sources`](https://newsapi.org/docs/endpoints/sources) endpoint:

In [ ]:
api.get_sources()

## Showtime

Let's get some news but using it together with OpenCV and Pandas


In [ ]:
import cv2
import cvzone
import screeninfo
import mediapipe as mp

import pandas as pd
import numpy as np

In [ ]:
# init
api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])

In [ ]:
def get_top_headlines():
    response = api.get_top_headlines(
        sources='bbc-news, cnn, fox-news, google-news, the-new-york-times, the-wall-street-journal, the-washington-post, time, usa-today, wired'
    )
    return response


top_headlines = get_top_headlines()

In [ ]:
print(
    """
    Top Headlines
    -------------
    {}
    """.format(top_headlines)
)

In [ ]:
def news_to_df(news):
    # map all the articles to a list
    articles = list(map(lambda x: x['title'], news['articles']))

    # create a dataframe
    df = pd.DataFrame(articles, columns=['title'])

    # add a column for the length of the title
    df['title_length'] = df['title'].apply(lambda x: len(x))

    # add a column for the number of words in the title
    df['title_words'] = df['title'].apply(lambda x: len(x.split(' ')))

    return df

In [ ]:
def news_to_df_all_cols(news):
    # map all the articles to a list, including all columns. Only for source column, we will take the name of the source
    articles = list(map(lambda x: [x['title'], x['description'], x['url'], x['urlToImage'], x['publishedAt'], x['source']['name']], news['articles']))

    # set column names
    columns = ['title', 'description', 'url', 'urlToImage', 'publishedAt', 'source']
    
    # create a dataframe
    df = pd.DataFrame(articles, columns=columns)

    return df

In [ ]:
df = news_to_df(top_headlines)

df.head()

In [ ]:
df = news_to_df_all_cols(top_headlines)

df.head()

### Simple WebCam Client directly consuming the NEWS API with blocking calls

In [ ]:
# Using OpenCV to display the image
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 60)

# text as one line string
text = ' '.join(df['title'].tolist())

# add '###' between each title
text = ' ... | '.join(df['title'].tolist())

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

zoomed_text_color = (0, 255, 0)
standard_text_color = (255, 255, 255)
color = standard_text_color

direction = 0
font_size = 12
box_size = 50

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():

        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.flip(image, 1)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a background for the text with a border
        cv2.rectangle(image, (0, image.shape[0] - box_size), (image.shape[1], image.shape[0]), (0, 0, 0), -1)
        # border
        cv2.rectangle(image, (0, image.shape[0] - box_size), (image.shape[1], image.shape[0]), (255, 255, 255), 2)


        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # mp_drawing.draw_landmarks(
                #     image,
                #     hand_landmarks,
                #     mp_hands.HAND_CONNECTIONS,
                #     mp_drawing_styles.get_default_hand_landmarks_style(),
                #     mp_drawing_styles.get_default_hand_connections_style())

                # if left hand is raised then move the text to the left
                if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.2 and len(results.multi_hand_landmarks) == 1:
                    text = text[1:] + text[0]
                    direction = 0
                    font_size = 12
                    color = standard_text_color
                    box_size = 50

                # if right hand is raised then move the text to the right
                elif hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x > 0.8 and len(results.multi_hand_landmarks) == 1:
                    direction = 1
                    font_size = 12
                    color = standard_text_color
                    box_size = 50

                # if both hands are raised then increase the font size to 36pt and change the color
                elif 0.2 < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.8 and len(results.multi_hand_landmarks) == 2:
                    font_size = 36
                    color = zoomed_text_color
                    box_size = 100

        else:
            # if no hands are detected then move the text to the left
            font_size = 12
            color = standard_text_color
            box_size = 50

        if direction == 0:
            text = text[1:] + text[0]
        elif direction == 1:
            text = text[-1] + text[:-1]

        # draw the text
        cv2.putText(image, text, (2, image.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, font_size / 12, color, 2)

        # put a logo in the top left corner
        logo = cv2.imread('logo.png')

        # resize the logo
        logo = cv2.resize(logo, (int(logo.shape[1] / 12), int(logo.shape[0] / 12)))

        # add the logo to the image
        image[0:logo.shape[0], 0:logo.shape[1]] = logo

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('AI News', image)

        # wait for the 'q' key to be pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # if 's' is pressed, save the image
        if cv2.waitKey(1) & 0xFF == ord('s'):
            cv2.imwrite('news.jpg', image)

cap.release()

# stop the video
cv2.destroyAllWindows()


## Building a server with NewsAPI and RabbitMQ

In [1]:
import json
import os
import time
from threading import Timer

import pandas as pd
import pika
from newsapi import NewsApiClient

from newsapi.newsapi_exception import NewsAPIException


class NewsPublisher:
    """
    A class to publish news articles to a RabbitMQ queue using the NewsAPI
    @param api_key: the api key to access the NewsAPI
    @param host: the host name of the RabbitMQ server
    @param queue_name: the name of the queue to publish the news articles
    """
    def __init__(self, api_key, host='localhost', queue_name='news_stream'):
        self.api = NewsApiClient(api_key=api_key)
        self.host = host
        self.queue_name = queue_name
        # set up a connection to RabbitMQ
        self.connection = pika.BlockingConnection(pika.ConnectionParameters(host=self.host))
        self.channel = self.connection.channel()
        self.sources = 'bbc-news, cnn, fox-news, google-news, time, wired, the-new-york-times, the-wall-street-journal, the-washington-post, usa-today, abc-news, associated-press, bloomberg, business-insider, cbs-news, cnbc, entertainment-weekly, espn, fortune, fox-sports, mtv-news, national-geographic, nbc-news, new-scientist, newsweek, politico, reddit-r-all, reuters, the-hill, the-huffington-post, the-verge, the-washington-times, vice-news'

        try:
            self.articles = self.api.get_everything(sources=self.sources)
            # save the news articles to a csv file
            _df = pd.DataFrame(self.articles['articles'])
            _df.to_csv('news.csv', index=False)
            _df.to_json('news.json', orient='records')
        except NewsAPIException as api_exception:
            print(f"Could not request results from NewsAPI; {api_exception}")
            print("Loading the news from the database...")

    def publish(self):
        """
        Publish the news articles to the RabbitMQ queue one by one and save the news to the database
        """
        try:
            for _article in self.articles['articles']:
                _body = json.dumps(_article).encode('utf-8')
                # add the news article to the queue
                self.channel.basic_publish(exchange='', routing_key=self.queue_name, body=_body)
        except NewsAPIException as api_exception:
            print(f"Could not request results from NewsAPI; {api_exception}")
            print("Loading the news from the database...")
            

        # do not close the connection until the message is delivered
        if self.connection.is_open:
            self.connection.close()

        # call the function again after 60 seconds
        Timer(60, self.publish).start()


    def destroy(self):
        """
        Close the connection to the RabbitMQ server
        """
        if self.connection.is_open:
            self.connection.close()


> Let's run a queue consumer that will be responsible for getting the news from the API and storing it in a database.

In [2]:
import os
import tkinter as tk
from tkinter import messagebox
import threading

class NewsPublisherGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("AI News")

        self.label = tk.Label(self.master, text="Click the button to start the AI News")
        self.label.pack()

        self.button = tk.Button(self.master, text="Start", command=self.start_publisher_thread)
        self.button.pack()

        self.api_key = os.environ.get('NEWSAPI_ORG')
        self.api = NewsPublisher(self.api_key)

        self.master.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_publisher_thread(self):
        t = threading.Thread(target=self.start_publisher)
        t.start()

    def start_publisher(self):
        print('Server is being initialized...')
        self.api.publish()
        self.button.config(text="Running...", state=tk.DISABLED)
        print('Server is now running...')

    def on_closing(self):
        if messagebox.askokcancel("Quit", "Do you want to quit? It will stop the NewsPublisher."):
            self.api.destroy()
            self.master.destroy()
            print('Server is being stopped by the user...')

    def run(self):
        self.master.geometry("320x80")
        self.master.mainloop()


In [3]:
def start_gui():
    root = tk.Tk()
    app = NewsPublisherGUI(root)
    app.run()

In [4]:
publisher_thread = threading.Thread(target=start_gui)
publisher_thread.start()

Server is being initialized...
Server is now running...


### News Consumer

> Let's run a queue consumer that will be responsible for getting the news from the API and storing it in a database.

In [ ]:
import pika
import json

class NewsConsumer:
    """
    A class to consume news articles from a RabbitMQ queue
    @param host: the host name of the RabbitMQ server
    @param queue_name: the name of the queue to consume the news articles
    """
    def __init__(self, host='localhost', queue_name='news_stream'):
        self.host = host
        self.queue_name = queue_name
        self.connection = pika.BlockingConnection(pika.ConnectionParameters(host=self.host))
        self.channel = self.connection.channel()
        self.channel.queue_declare(queue=self.queue_name)
        self.channel.basic_consume(queue=self.queue_name, on_message_callback=self.callback, auto_ack=True)
        
    def callback(self, ch, method, properties, body):
        """
        A callback function that is called whenever a new message is received
        @param ch: the channel object
        @param method: a method object
        @param properties: message properties
        @param body: message body (a news article in JSON format)
        """
        article = json.loads(body)
        print(f"Received a news article from {article['source']['name']}: {article['title']}")
        
    def consume(self):
        """
        Start consuming messages from the RabbitMQ queue
        """
        print("Starting to consume news articles...")
        self.channel.start_consuming()
        
    def destroy(self):
        """
        Close the connection to the RabbitMQ server
        """
        if self.connection.is_open:
            self.connection.close()


In [3]:
# generate GUI for the user to start the news publisher
import tkinter as tk
from tkinter import messagebox
import pika

# create a window
cwin = tk.Tk()
cwin.title("AI News")

# create a label
label = tk.Label(cwin, text="Click the button to consume published news..")
# create a button without the command to start the news publisher
button = tk.Button(cwin, text="Start")


_host = 'localhost'
consumer = pika.BlockingConnection(pika.ConnectionParameters(_host))
channel = consumer.channel()
channel.queue_declare(queue='news_stream')

# start the news publisher
def start_consumer():
    print('Consumer is being initialized...')

    # create a function which is called on incoming messages
    def callback(ch, method, properties, body):
        print(" [x] Received %r" % body)

    # set up subscription on the queue
    channel.basic_consume(queue='news_stream', on_message_callback=callback, auto_ack=True)

    # start consuming (blocks)
    channel.start_consuming()

    # change the button text to 'Stop'
    button.config(text="Running...", state=tk.DISABLED)

    print('Server is now running...')

def on_closing():
    if messagebox.askokcancel("Quit", "Do you want to quit? It will stop the NewsPublisher."):
        cwin.destroy()
        print(
            'Server is being stopped by the user...'
        )

# add the command to the button
button.config(command=start_publisher)

# add on closing event listener
cwin.protocol("WM_DELETE_WINDOW", on_closing)

# place the label and the button on the window
label.pack()
button.pack()

# set the window size
cwin.geometry("320x80")


# start the GUI
cwin.mainloop()

Server is being initialized...


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\yilma\OneDrive\Apps\Local\apps\anaconda3\2023.03\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\yilma\AppData\Local\Temp\ipykernel_28460\551332361.py", line 25, in start_publisher
    api.publish()
  File "C:\Users\yilma\AppData\Local\Temp\ipykernel_28460\1570467588.py", line 47, in publish
    self.channel.basic_publish(exchange='', routing_key=self.queue_name, body=_body)
  File "c:\Users\yilma\.virtualenvs\aij-L0AtPH4U\lib\site-packages\pika\adapters\blocking_connection.py", line 2259, in basic_publish
    self._impl.basic_publish(
  File "c:\Users\yilma\.virtualenvs\aij-L0AtPH4U\lib\site-packages\pika\channel.py", line 423, in basic_publish
    self._raise_if_not_open()
  File "c:\Users\yilma\.virtualenvs\aij-L0AtPH4U\lib\site-packages\pika\channel.py", line 1403, in _raise_if_not_open
    raise exceptions.ChannelWrongStateError('Channel is closed.')
pika

Server is being stopped by the user...
